### USING PCA TO ACHIEVE SEPARABILITY

In [ ]:
#https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

%matplotlib inline

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
# load dataset into Pandas DataFrame
df = pd.read_csv(url, names=['sepal length','sepal width','petal length','petal width','target'])


### Looking at the Data

In [ ]:
#First look at the data
df.info()

In [ ]:
df.head()

In [ ]:
df['target'].unique()

In [ ]:
df.describe()

In [ ]:
sns.heatmap(df.isna(),cbar=False)

In [ ]:
corrMat = df.corr()
ax =sns.heatmap(corrMat,vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

In [ ]:
features = ['sepal length', 'sepal width', 'petal length', 'petal width']
# Separating out the features
x = df.loc[:, features].values
# Separating out the target
y = df.loc[:,['target']].values

In [ ]:
X = StandardScaler().fit_transform(x)

In [ ]:
(x[:10],X[:10])

In [ ]:
print(f"Before standard transofrmation:")
print((min(x[:][0]),max(x[:][0])))
print(f"After standard transofrmation:")
print((min(X[:][0]),max(X[:][0])))

In [ ]:
pca = PCA(2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data=principalComponents,columns=['PC1','PC2'])
principalDf.shape
#Add target column
finalDf = pd.concat([principalDf, df[['target']]], axis = 1)

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
colors = ['r', 'g', 'b']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['target'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'PC1']
               , finalDf.loc[indicesToKeep, 'PC2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

### LESS SEPARATED BEFORE PCA

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel(features[0], fontsize = 15)
ax.set_ylabel(features[1], fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
colors = ['r', 'g', 'b']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['target'] == target
    ax.scatter(df.loc[indicesToKeep, features[0]]
               , df.loc[indicesToKeep, features[1]]
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel(features[-1], fontsize = 15)
ax.set_ylabel(features[-2], fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
colors = ['r', 'g', 'b']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['target'] == target
    ax.scatter(df.loc[indicesToKeep, features[-1]]
               , df.loc[indicesToKeep, features[-2]]
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

In [ ]:
print(f'Amount of variance explained by PC1 and PC2 {sum(pca.explained_variance_ratio_)}')

### PCA TO SPEED UP LEARNING

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')

In [ ]:
mnist.data.shape

In [ ]:
#choose the minimum number of principal components such that 95% of the variance is retained
pca = PCA(.95)

In [ ]:
from sklearn.model_selection import train_test_split
train_img, test_img, train_lbl, test_lbl = train_test_split( mnist.data, mnist.target, test_size=1/7.0, random_state=0)

In [ ]:
scaler = StandardScaler()
# Fit on training set only.
scaler.fit(train_img)
# Apply transform to both the training set and the test set.
train_img = scaler.transform(train_img)
test_img = scaler.transform(test_img)

In [ ]:
pca.fit(train_img)

In [ ]:
train_img = pca.transform(train_img)
test_img = pca.transform(test_img)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logisticRegr = LogisticRegression(solver = 'lbfgs')

In [ ]:
logisticRegr.fit(train_img, train_lbl)

In [ ]:
logisticRegr.predict(test_img[0].reshape(1,-1))

In [ ]:
logisticRegr.predict(test_img[0:10])

In [ ]:
logisticRegr.score(test_img, test_lbl)

### PCA without SKLEARN

In [ ]:
df.head()

In [ ]:
X = df[['sepal length','sepal width', 'petal length' , 'petal width']].to_numpy()

In [ ]:
X.shape
N = X.shape[0]

In [ ]:
X_mean=X.mean(axis=0).reshape((1,4))

In [ ]:
import numpy as np
S=np.zeros((4,4))
for n in range(N):
    T = X[n]-X_mean
    S+=np.dot(T.T,T)
S/=N
# T = X-X_mean
# S = np.dot(T.T,T)/N
S.shape

In [ ]:
eigVals = np.linalg.eig(S)[0]
eigVectors = np.linalg.eig(S)[1]

In [ ]:
sample_variance = X.var(axis=0).sum()
sample_variance

In [ ]:
eigVals.sum()

In [ ]:
PC1 = eigVectors[:,0].reshape((4,1))
PC2 = eigVectors[:,1].reshape((4,1))
PC3 = eigVectors[:,2].reshape((4,1))
PC4 = eigVectors[:,3].reshape((4,1))

In [ ]:
eigVectors

In [ ]:
X.shape

In [ ]:
PC = np.c_[PC1,PC2]
X_transformed = np.dot(X,PC)
X_transformed.shape

In [ ]:
plt.scatter(X_transformed[:,0],X_transformed[:,1])
plt.show()

In [ ]:
PC_v2 = np.c_[PC3,PC4]
X_transformed_v2 = np.dot(X,PC_v2)
X_transformed_v2.shape

In [ ]:
plt.scatter(X_transformed_v2[:,0],X_transformed_v2[:,1])
plt.show()

### Compare with LDA - for class separability

In [ ]:
labels = df['target'].unique()
labels

In [ ]:
#Get rows corresponding to each class
idx0 = df['target']==labels[0]
idx1 = df['target']==labels[1]
idx2 = df['target']==labels[2]

#Filtering out the data corresponding the 3 classes
X0 = X[idx0]
X1 = X[idx1]
X2 = X[idx2]
print(X0.shape,X1.shape,X2.shape)

In [ ]:
mTotal = X.mean(axis=0).reshape((1,4))
m0 = X0.mean(axis=0).reshape((1,4))
m1 = X1.mean(axis=0).reshape((1,4))
m2 = X2.mean(axis=0).reshape((1,4))

In [ ]:
T0 = X0-m0
S0 = np.dot(T0.T,T0)

T1 = X1-m1
S1 = np.dot(T1.T,T1)

T2 = X2-m2
S2 = np.dot(T2.T,T2)

S_W = S0+S1+S2
print(S_W.shape)

T_total = X-mTotal
S_T = np.dot(T_total.T,T_total)
print(S_T.shape)

S_B = S_T-S_W

S_B

In [ ]:
S = np.dot(np.linalg.inv(S_W),S_B)
eigVals = np.linalg.eig(S)[0]
eigVectors = np.linalg.eig(S)[1]

In [ ]:
LD1 = eigVectors[:,0].reshape((4,1))
LD2 = eigVectors[:,1].reshape((4,1))
LD3 = eigVectors[:,2].reshape((4,1))
LD4 = eigVectors[:,3].reshape((4,1))

In [ ]:
eigVectors

In [ ]:
LDC = np.c_[LD1,LD2]
X_transformed_LDA = np.dot(X,LDC)
X_transformed_LDA.shape

In [ ]:
plt.scatter(X_transformed_LDA[:,0],X_transformed_LDA[:,1])
plt.show()

### LDA with sklearn

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from sklearn import datasets
iris = datasets.load_iris()

def encode(s):
    L = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
    return L.index(s)

y = df['target'].apply(encode)
target_names = (df['target'].unique()).tolist()

pca = PCA(n_components=2)
X_r = pca.fit(X).transform(X)

lda = LinearDiscriminantAnalysis(n_components=2)
X_r2 = lda.fit(X, y).transform(X)

# Percentage of variance explained for each components
print('explained variance ratio (first two components): %s'
      % str(pca.explained_variance_ratio_))

plt.figure()
colors = ['navy', 'turquoise', 'darkorange']
lw = 2

for color, i, target_name in zip(colors, [0, 1, 2], target_names):
    plt.scatter(X_r[y == i, 0], X_r[y == i, 1], color=color, alpha=.8, lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('PCA of IRIS dataset')

plt.figure()
for color, i, target_name in zip(colors, [0, 1, 2], target_names):
    plt.scatter(X_r2[y == i, 0], X_r2[y == i, 1], alpha=.8, color=color,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('LDA of IRIS dataset')

plt.show()